<a href="https://colab.research.google.com/github/yatishdurga/AI_AGENTS_USING_LLMs/blob/main/Q%26A_Agent_with_RAG_ChomaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0

In [ ]:
!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0
# Access operating system functionalities (e.g., paths, environment variables)
import os
# Load and extract text content from unstructured files (PDF, .docx, etc.)
from langchain.document_loaders import UnstructuredFileLoader
# Used to split large documents into manageable chunks of text
from langchain_text_splitters import CharacterTextSplitter
# Load pre-trained Hugging Face embeddings to convert text into vector form

from langchain_community.embeddings import HuggingFaceEmbeddings
# Interface to store and retrieve vector embeddings using ChromaDB

from langchain_chroma import Chroma
# Integrate Groq's LLM (e.g., using Mixtral) for fast inference

from langchain_groq import ChatGroq
# Create a QA system by combining document retrieval and LLM for answering queries

from langchain.chains import RetrievalQA

In [ ]:
os.environ["GROQ_API_KEY"]=""

In [ ]:
!pip install pymupdf langchain-community
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("/content/attention-is-all-you-need.pdf")
documents = loader.load()

# Optional: Check first 500 characters
print(documents[0].page_content[:500])

Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.com
Aidan N. Gomez∗†
University of Toronto
aidan@cs.toronto.edu
Łukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or


In [ ]:
print(documents)

[Document(metadata={'source': '/content/attention-is-all-you-need.pdf', 'file_path': '/content/attention-is-all-you-need.pdf', 'page': 0, 'total_pages': 11, 'format': 'PDF 1.3', 'title': 'Attention is All you Need', 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'keywords': '', 'creator': '', 'producer': 'PyPDF2', 'creationDate': '', 'modDate': "D:20180212212210-08'00'", 'trapped': ''}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nA

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

In [ ]:
texts=text_splitter.split_documents(documents)


In [ ]:
type(texts)

list

In [ ]:
len(texts)

11

In [ ]:
texts[:3]

[Document(metadata={'source': '/content/attention-is-all-you-need.pdf', 'file_path': '/content/attention-is-all-you-need.pdf', 'page': 0, 'total_pages': 11, 'format': 'PDF 1.3', 'title': 'Attention is All you Need', 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'keywords': '', 'creator': '', 'producer': 'PyPDF2', 'creationDate': '', 'modDate': "D:20180212212210-08'00'", 'trapped': ''}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nA

In [ ]:
embeddings = HuggingFaceEmbeddings()

<ipython-input-11-d0c9174021d8>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as se

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
persistant_directory="doc_db"

In [ ]:
vectordb= Chroma.from_documents(
    documents=texts,
    embedding=embeddings,
    persist_directory=persistant_directory
)


In [ ]:
# retriever
retriever=vectordb.as_retriever

In [ ]:
#llm from groq
llm=ChatGroq(
    model="llama3-70b-8192",
    temperature=0
)

In [ ]:

# 1. Load embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Load documents into Chroma vectorstore
db = Chroma.from_documents(documents, embedding_model)

# 3. Create retriever from Chroma vectorstore
retriever = db.as_retriever()

# 4. Load LLM (Groq)
llm = ChatGroq(temperature=0, model_name="llama3-8b-8192")

# 5. Create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,  # ✅ must be a retriever, not a dict
    return_source_documents=True
)


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,  # ✅ must be a retriever, not a dict
    return_source_documents=True
)

In [ ]:
# Ask the question using the QA chain
query = "What is the main motto of the paper? How is it useful for the data science field?"

response = qa_chain.invoke({"query": query})


In [ ]:
print(response["result"])
print("**"*15)
print(response["source_documents"][0].metadata["source"])

The main motto of the paper is to introduce the Transformer, a novel sequence-to-sequence model that replaces the traditional recurrent neural network (RNN) and convolutional neural network (CNN) architectures with self-attention mechanisms. The Transformer is designed to handle long-range dependencies and parallelize the processing of input sequences, making it more efficient and effective for machine translation tasks.

The Transformer is useful for the data science field in several ways:

1. **Efficient processing of long sequences**: The Transformer can process input sequences of arbitrary length, making it suitable for tasks such as machine translation, text summarization, and language modeling.
2. **Parallelization**: The Transformer's self-attention mechanism allows it to parallelize the processing of input sequences, making it much faster than traditional RNN and CNN architectures.
3. **Improved performance**: The Transformer has been shown to achieve state-of-the-art results o